# Análisis modelo mercado canasta usando la base de datos "Instacart Online Grocery" y Spark + FP-growth

Para esta libreta, se usará la base de datos abierta de [Instacart](https://www.kaggle.com/c/instacart-market-basket-analysis/data) que contiene más de 3 millones de transacciones.

> "Instacart" es una empresa estadounidense que opera un servicio de entrega y recogida de comestibles. La empresa ofrece sus servicios a través de un sitio web y una aplicación móvil.
La BD contiene más de 3 millones de transacciones provenientes de más de 200,000 usuarios (todos los datos peronales está anonimizados). Esta BD también describe la hora y fecha en que se realizó el pedido

En este libreta, usaremos el algoritmo FP-growth para encontrar cuáles son los productos que se compran normalmente juntos, a partir del cálculo de soporte e identificación de las reglas de asociación.
<img src="https://s3.us-east-2.amazonaws.com/databricks-dennylee/media/buy+it+again+or+recommend.png" width="1100"/>

![](https://s3.us-east-2.amazonaws.com/databricks-dennylee/media/data-engineering-pipeline-3.png)

# Carga de datos

Primero, debemos descargar los datos disponibles en [Kaggle](https://www.kaggle.com/c/instacart-market-basket-analysis/data) y cargarlos en la opción ADD DATA. 
De manera general, los archivos contienen la siguiente información:

* `Orders`: +3.4 millones de filas, 206K usuarios
* `Products`: +50 registros
* `Aisles`: 134 registros 
* `Departments`: 21 registros
* `order_products__SET`:
  * `train`: 131,000 órdenes
  
[Para mayor descripción de los datos](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b)

In [0]:
%fs ls /FileStore/tables/

path,name,size
dbfs:/FileStore/tables/2019.csv,2019.csv,8510
dbfs:/FileStore/tables/ArtOfWar.txt,ArtOfWar.txt,63371
dbfs:/FileStore/tables/aisles.csv,aisles.csv,2603
dbfs:/FileStore/tables/amazon_alexa.csv,amazon_alexa.csv,514749
dbfs:/FileStore/tables/amazon_alexa.tsv,amazon_alexa.tsv,514752
dbfs:/FileStore/tables/arte_guerra.txt,arte_guerra.txt,63371
dbfs:/FileStore/tables/books.xml,books.xml,5542
dbfs:/FileStore/tables/data_fraudes.csv,data_fraudes.csv,146558829
dbfs:/FileStore/tables/departments.csv,departments.csv,270
dbfs:/FileStore/tables/fraudes_data.csv,fraudes_data.csv,493534783


In [0]:
#Importamos los datos
aisles = spark.read.csv("/FileStore/tables/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("/FileStore/tables/departments.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/FileStore/tables/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/FileStore/tables/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/FileStore/tables/products.csv", header=True, inferSchema=True)

#Creamos tablas temporales
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

# Análisis exploratorio de los datos usando Spark SQL

In [0]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from orders  
 group by order_dow 
 order by total_orders desc

total_orders,day_of_week
600905,Sunday
587478,Monday
467260,Tuesday
453368,Friday
448761,Saturday
436972,Wednesday
426339,Thursday


In [0]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from orders 
 group by order_hour_of_day 
 order by order_hour_of_day

total_orders,hour
22758,0
12398,1
7539,2
5474,3
5527,4
9569,5
30529,6
91868,7
178201,8
257812,9


In [0]:
%sql
select countbydept.*
  from (
  -- de la tabla productos, contamos el número de registros por depto
  -- y los ordenamos (del mayor al menor)
  select department_id, count(1) as counter
    from products
   group by department_id
   order by counter asc 
  ) as maxcount
inner join (
  -- se repite el procedimiento, pero ahora se unen las tablas productos y deptos para obtener una lista completa
  select
    d.department_id,
    d.department,
    count(1) as products
    from departments d
      inner join products p
         on p.department_id = d.department_id
   group by d.department_id, d.department 
   order by products desc
  ) countbydept 
  -- combinamos las 2 consultas, para contar el número de productos por depto
  on countbydept.products = maxcount.counter

department_id,department,products
16,dairy eggs,3449
14,breakfast,1115
3,bakery,1516
21,missing,1258
6,international,1139
1,frozen,4007
11,personal care,6563
9,dry goods pasta,1858
13,pantry,5371
19,snacks,6264


In [0]:
%sql
select d.department, count(distinct p.product_id) as products
  from products p
    inner join departments d
      on d.department_id = p.department_id
 group by d.department
 order by products desc
 limit 10

department,products
personal care,6563
snacks,6264
pantry,5371
beverages,4365
frozen,4007
dairy eggs,3449
household,3084
canned goods,2092
dry goods pasta,1858
produce,1684


## Analizando la canasta de compra

In [0]:
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [0]:
display(baskets)

order_id,items
1139,"List(Cinnamon Rolls with Icing, Red Vine Tomato, Picnic Potato Salad, Flaky Biscuits, Organic Strawberries, Organic Bakery Hamburger Buns Wheat - 8 CT, Buttermilk Biscuits, Banana, Guacamole)"
1143,"List(Water, Natural Premium Coconut Water, Organic Red Radish, Bunch, Organic Capellini Whole Wheat Pasta, Organic Raspberries, Calming Lavender Body Wash, Organic Garlic, Rustic Baguette, Organic Brussel Sprouts, Organic Butterhead (Boston, Butter, Bibb) Lettuce, Organic Blueberries, Spring Water, Large Lemon, Basil Pesto, Baby Arugula, Organic Hass Avocado, Unscented Long Lasting Stick Deodorant)"
1342,"List(Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas)"
1468,"List(Pomegranate Seeds, Organic Red Radish, Bunch, Natural Mini Pork Pepperoni, Cage Free Grade AA Large White Eggs, Bartlett Pears, Organic Red Potato, Organic Ginger Root, Banana, Red Peppers, Active Dry Yeast, Organic Lacinato (Dinosaur) Kale, Organic Baby Broccoli, Carrots, Fresh Cauliflower, Organic English Cucumber, Organic Grape Tomatoes, Organic Hass Avocado)"
1591,"List(Cracked Wheat, Strawberry Rhubarb Yoghurt, Organic Bunny Fruit Snacks Berry Patch, Goodness Grapeness Organic Juice Drink, Honey Graham Snacks, Spinach, Granny Smith Apples, Oven Roasted Turkey Breast, Pure Vanilla Extract, Chewy 25% Low Sugar Chocolate Chip Granola, Banana, Original Turkey Burgers Smoke Flavor Added, Twisted Tropical Tango Organic Juice Drink, Navel Oranges, Lower Sugar Instant Oatmeal Variety, Ultra Thin Sliced Provolone Cheese, Natural Vanilla Ice Cream, Cinnamon Multigrain Cereal, Garlic, Goldfish Pretzel Baked Snack Crackers, Original Whole Grain Chips, Medium Scarlet Raspberries, Lemon Yogurt, Original Patties (100965) 12 Oz Breakfast, Nutty Bars, Strawberry Banana Smoothie, Green Machine Juice Smoothie, Coconut Dreams Cookies, Buttermilk Waffles, Uncured Genoa Salami, Organic Greek Whole Milk Blended Vanilla Bean Yogurt)"
1721,"List(Organic Thompson Seedless Raisins, Whole Milk Plain Yogurt, Tomato Paste, Organic Rolled Oats, Organic Reduced Fat Milk, Organic Whole Milk)"
2711,"List(Granny Smith Apples, Alpine Spring Water, Mountain Spring Water, Honeycrisp Apples)"
2888,"List(Shredded Sharp Cheddar Cheese, Organic Raspberries, Clover Honey, Cherubs Heavenly Salad Tomatoes, Whipped Cream Cheese Spread, Ground Cumin, 100% Whole Wheat Bread, Pure & Natural Sour Cream, Organic Milk Whole, Pizza Sauce, 50% Less Sodium Black Beans, Creamy Almond Butter, Organic Grade A Large Brown Eggs, Boneless And Skinless Chicken Breast, Celery Hearts, Low Moisture Part Skim Mozzarella String Cheese, Whole Chicken, Organic Strawberries, Almonds Roasted No Salt, Organic Large Extra Fancy Fuji Apple, Organic Dried Sweetened Cranberries, Whole Kernel Corn Golden Sweet, Pecan Chips, Pepperoni, Extra Virgin Olive Oil, Hass Avocados, Super Soft Taco Flour Tortillas, Natural Finely Shredded Triple Cheddar Cheese, Grass Fed Angus Ground Beef 93/7, Sunflower Nuts Roasted/No Salt, 100% Pure Vegetable Oil, Mandarins, Greek Vanilla Yogurt, Cinnamon Raisin Bagels)"
3179,"List(Caramel Almond and Sea Salt Nut Bar, White Chocolate Macadamia Nut Energy Bar, Tea Tree Scalp Treatment Shampoo, Sparkling Water Grapefruit, Nuts & Spices, Dark Chocolate Mocha Almond, Honey Bunny Grahams, Lime Sparkling Water, Krinkle Cut Classic Barbecue Potato Chips)"
4092,"List(Organic Baby Spinach, Sharp Cheddar Cheese, Garlic, Mozzarella Cheese, Organic Zucchini, Organic Extra Firm Tofu, Organic Garbanzo Beans, Italian Extra Virgin Olive Oil, Salted Butter, Mint, Kale Greens, Large Lemon, Carrots, Limes, Organic Grape Tomatoes, Parsley, Italian (Flat), New England Grown, Fresh Ginger Root)"


# Entrenando el modelo

Vamos a analizar qué elementos son comprados en conjunto en la misma transacción (pañales, cerveza). Para lograr esto implementaremos un análisis de modelo mercado canasta usando el algoritmo de minería de elementos frecuentes [FP-growth](https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html#fp-growth).

In [0]:
%scala
import org.apache.spark.ml.fpm.FPGrowth

// Extraemos los elementos
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Aqui: usamos FPGrowth :D
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_e87cb612395b
model: org.apache.spark.ml.fpm.FPGrowthModel = FPGrowthModel: uid=fpgrowth_e87cb612395b, numTrainingRecords=131209

In [0]:
%scala
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]

In [0]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",710
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",649
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",587
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",531
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",497
"List(Organic Avocado, Organic Baby Spinach, Banana)",484
"List(Organic Avocado, Large Lemon, Banana)",477
"List(Limes, Large Lemon, Banana)",452
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",424
"List(Limes, Organic Avocado, Large Lemon)",389


# Revisando las reglas de asociación

Una de las ventajas de `FP-growth` es que además de indicar los elementos más frecuentes, también generar las reglas de asociación. Por ejemplo, si un comprador compra mantequilla de maní, ¿cuál es la probabilidad de que también compre mermelada?. Para mayor información [A Gentle Introduction on Market Basket Analysis — Association Rules](https://towardsdatascience.com/a-gentle-introduction-on-market-basket-analysis-association-rules-fa4b986a40ce)

In [0]:
%scala
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 2 more fields]

In [0]:
%sql
select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 50

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5984251968503937
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.546875
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5459770114942529
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5412186379928315
"List(Yellow Onions, Strawberries)",List(Banana),0.5357142857142857
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5314685314685315
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.521099116781158
"List(Organic D'Anjou Pears, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5170454545454546
"List(Organic Unsweetened Almond Milk, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5141065830721003
